In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix

In [2]:
df_original = pd.read_csv("/mnt/data/public/bgg/bgg-19m-reviews.csv")

# EDA

In [3]:
df = df_original.copy()
df = df.iloc[:,1:]

In [4]:
df.isna().sum()

user             66
rating            0
comment    15596189
ID                0
name              0
dtype: int64

In [5]:
df.dropna(subset = ["user"], inplace = True)

In [6]:
duplicates = df[df.duplicated(subset=["user", "name"], keep=False)]
duplicates.sort_values(by="user").tail(2)

,user,rating,comment,ID,name
1916530,zzzabiss,8.0,Va muy bien como un filler introductorio. He r...,129622,Love Letter
12089270,zzzabiss,8.0,NaN,277085,Love Letter


In [7]:
df_description = pd.read_csv("/mnt/data/public/bgg/games_detailed_info.csv")

/tmp/ipykernel_354/2684239534.py:1: DtypeWarning: Columns (29,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_description = pd.read_csv("/mnt/data/public/bgg/games_detailed_info.csv")


In [8]:
print(df_description[df_description["id"] == 129622]['alternate'])
print(df_description[df_description["id"] == 277085]['alternate'])

17    ['Letters to Santa', 'List Miłosny', 'Lista Sk...
Name: alternate, dtype: object
1057    ['List miłosny (Edycja Premium)', 'Love Letter...
Name: alternate, dtype: object


they just save the common names under names, but same name have different editions (example, the two rows below are the regular vs premium edition)

based on my opinion, i suggest to sort by item ID since premium editions may come at different price points/qualities

In [9]:
duplicates = df[df.duplicated(subset=["user", "ID"], keep=False)]

if we sort by ID though, no duplicates

# Make Utility Matrix

In [10]:
df = df[["user","rating","ID"]]
df = df.rename(columns={"user": "user_name", 
                        "ID": "item_id"})

In [11]:
# map users to unique interger row indices
user_mapping = pd.Categorical(df['user_name'])
user_indices = user_mapping.codes
user_dict = dict(enumerate(user_mapping.categories))

# map items to unique interger col indices
item_mapping = pd.Categorical(df['item_id'])
item_indices = item_mapping.codes
item_dict = dict(enumerate(item_mapping.categories))

# build sparse matrix
df_utility = csr_matrix((df['rating'], (user_indices, item_indices)))

In [12]:
df_utility

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 18964741 stored elements and shape (412815, 21839)>

# ALS

In [23]:
def als(M_arr, d, tol):
    M_csr = M_arr.copy()
    M_csc = M_arr.tocsc()
    

    n, k = M_arr.shape

    # Initialize U and V matrices with ones
    U = np.ones((n, d))
    V = np.ones((k, d))

    while True:
        # Fix U and solve for V (each item independently)
        print("Fix U and solve V")
        for j in range(k):
            users = M_csc[:, j].indices
            if users.size > 0:
                A = U[users]
                b = M_csc[users, j].toarray().flatten()
                try:
                    V[j, :] = np.linalg.solve(A.T @ A, A.T @ b)
                except np.linalg.LinAlgError:
                    V[j, :] = np.linalg.lstsq(A, b, rcond=None)[0]

        print("Fix V and solve U")
        # Fix V and solve for U (each user independently)
        for i in range(n):
            items = M_csr[i, :].indices
            if items.size > 0:
                A = V[items]
                b = M_csr[i, items].toarray().flatten()
                try:
                    U[i, :] = np.linalg.solve(A.T @ A, A.T @ b)
                except np.linalg.LinAlgError:
                    U[i, :] = np.linalg.lstsq(A, b, rcond=None)[0]

        # Calculate current predictions
        pred = U @ V.T

        # Calculate RMSE for observed ratings (use NumPy arrays)
        rmse = np.sqrt(np.mean((M_arr[mask] - pred[mask]) ** 2))

        # Check for convergence
        if rmse < tol:
            break
        else:
            break

    return U, V

In [ ]:
f_user, f_item = als(df_utility, 50, 2)

Fix U and solve V
